# Lord of the Rings Lore Master

This notebook creates a chat bot which is an expert on Lord of the Rings lore. It uses Retrieval Augmented Generation (RAG) on the LotR trilogy.

In [1]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

from langchain import hub
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing_extensions import List, TypedDict
from langchain_core.prompts import ChatPromptTemplate

## Model/Embedding/Vector Store Choices

In [2]:
# select chat model
llm = ChatOpenAI(model="gpt-4o-mini")

# select embedding model
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# select vector store
vector_store = InMemoryVectorStore(embeddings)

## Parsing the LotR Trilogy!

In [3]:
# read the trilogy into a long string
trilogy = ''

# Load The Fellowship of the Ring
with open("../data/LotR/FotR.txt") as f:
    trilogy += f.read()

# Load The Two Towers
with open("../data/LotR/TT.txt") as f:
    trilogy += f.read()

# Load The Return of the King
with open("../data/LotR/RotK.txt") as f:
    trilogy += f.read()

print(len(trilogy))

2582402


In [4]:
# split into chunks of size 1000 with overlap of 200
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
text_chunks = text_splitter.create_documents([trilogy])

print(f"Split trilogy into {len(text_chunks)} chunks.")

Split trilogy into 3683 chunks.


In [5]:
# print some info about a chunk
chunk = text_chunks[40]

print(type(chunk))
print(chunk.metadata)
print(len(chunk.page_content))
print(chunk.page_content[:102])

<class 'langchain_core.documents.base.Document'>
{'start_index': 26626}
993
It is probable that the craft of building, as many other crafts beside, was derived from the DÃºnedain


## Store Text Chunks in a Vector Store

The text embedding model converts text into vector embeddings which we then save in a vector store.

In [6]:
document_ids = vector_store.add_documents(documents=text_chunks)

print(document_ids[:3])

['0b6e2a45-dad8-4c4a-9e03-a06d82f5661b', 'e22dea32-3c52-4add-8950-7c1c761d84e2', '5b857347-26c9-49de-9eb0-486106b529b4']


## Retrieval and Generation

In [7]:
# load a RAG prompt from LangChain's hub
prompt = hub.pull("rlm/rag-prompt")

# this prompt expects a question and some context, as seen in this example
example_messages = prompt.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

assert len(example_messages) == 1
print(example_messages[0].content)

c:\Users\alexk\Projects\.venv\lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: (question goes here) 
Context: (context goes here) 
Answer:


In [8]:
# here is a python class to hold a question/context/answer triplet
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [9]:
# here are the retrieval and generation functions

def retrieve(state: State):
    '''search vector store for context related to question and fill state's context'''
    retrieved_docs = vector_store.similarity_search(state["question"])
    state["context"] = retrieved_docs

def generate(state: State):
    '''add question and context to hub's RAG prompt and invoke llm'''
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    state["answer"] = response.content

In [10]:
# and here we try it out!
my_state = State()
my_state["question"] = "What is a Palantir?"

# retrieve the context and print it out to see what the embedding model focuses on
retrieve(my_state)
print(my_state)

{'question': 'What is a Palantir?', 'context': [Document(id='196af002-41f4-45ed-b836-fbd52652dd5d', metadata={'start_index': 1465814}, page_content='_Chapter 11_\n            The PalantÃ\xadr'), Document(id='1312deb2-151a-44c9-b396-79adb38f9bb2', metadata={'start_index': 1492454}, page_content="'Each _palantÃ\xadr_ replied to each, but all those in Gondor were ever open to the view of Osgiliath. Now it appears that, as the rock of Orthanc has withstood the storms of time, so there the _palantÃ\xadr_ of that tower has remained. But alone it could do nothing but see small images of things far off and days remote. Very useful, no doubt, that was to Saruman; yet it seems that he was not content. Further and further abroad he gazed, until he cast his gaze upon Barad-dÃ»r. Then he was caught!\n     'Who knows where the lost Stones of Arnor and Gondor now lie buried, or drowned deep? But one. at least Sauron must have obtained and mastered to his purposes. I guess that it was the Ithil-stone,

In [11]:
# now ask the LLM the question with context!
generate(my_state)
print(my_state["answer"])

A Palantir, or _palantír_, is a seeing stone that allows the user to view distant places and events. These stones were created by the Noldor, possibly by Fëanor, and are capable of being misused for evil purposes, as seen with Saruman and Sauron. The term itself means "that which looks far away."


## Use a Custom Prompt Template to Influence the Voice

In [12]:
# set up a prompt template 
elven_sage_template = ChatPromptTemplate.from_messages([
    ("user", "You are an Elven sage who is an expert on the history of Middle Earth. \
     Use the following pieces of retrieved context to answer the question. \
     If you don't know the answer, just say that you don't know. \
     Make your language sound like it was written by Tolkien.\
     \nQuestion: {question} \
     \nContext: {context} \
     \nAnswer:")
])

# make a new generate function with this template
def generate_with_template(state: State, template: ChatPromptTemplate):
    '''add question and context to hub's RAG prompt and invoke llm'''
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = template.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    state["answer"] = response.content

In [13]:
# let's see how this changes the answer
my_state = State()
my_state["question"] = "What is a Palantir?"

# retrieve the context
retrieve(my_state)

# ask the question with context
generate_with_template(my_state, elven_sage_template)
print(my_state["answer"])

Ah, dear seeker of knowledge, the Palantir, or _palantíri_ as it is known in the ancient tongue, is a device of great mystery and power, wrought by the hands of the Noldor in the days of yore, perhaps even by the fabled Fëanor himself. The name itself doth mean "that which looks far away," for these seeing-stones were crafted to perceive the distant realms and the events transpiring therein. 

Each Palantir is a sphere of great artistry, capable of revealing images of far-off places and times, yet they are not without peril, for the gaze of one may be ensnared by another. Such was the fate of Saruman, who, in his hubris, sought to wield the power of the Orthanc-stone, only to find himself ensnared in the web of Sauron’s malice.

In the twilight of the Third Age, these stones became relics of a bygone age, scattered and lost to the annals of time. The remnants of their glory lie buried in the histories of Gondor and Arnor, remembered only in the whispers of lore and the secretive murmur